# Scatter3d plot with slider

In this example, we combine a three-dimensional scatter plot with a slider,
which is used to navigate the time dimension of our data.

In [ ]:
import plopp as pp
import plopp.widgets as pw
import scipp as sc
import numpy as np

We first generate some data that represents events detected on a cylindrical detector panel,
as a function of time.

In [ ]:
nphi = 100
nz = 20
nt = 50

r = sc.scalar(10.0, unit='m')
phi = sc.linspace('phi', 0, np.pi, nphi, unit='rad')
z = sc.linspace('z', -3.0, 3.0, nz, unit='m')
t = sc.linspace('time', 0.0, 6.0, 50, unit='s')

x = r * sc.cos(phi)
y = r * sc.sin(phi)

h, _ = np.histogramdd(np.random.standard_normal(size=[10_000, 3]), bins=(nt, nz, nphi))
a = sc.array(dims=['time', 'z', 'phi'], values=h, unit='counts')
sizes = a.sizes.copy()
del sizes['time']

da = sc.DataArray(
    data=a.flatten(dims=['z', 'phi'], to='pixel'),
    coords={
        'x': sc.broadcast(x, sizes=sizes).flatten(to='pixel'),
        'y': sc.broadcast(z, sizes=sizes).flatten(to='pixel'),
        'z': sc.broadcast(y, sizes=sizes).flatten(to='pixel'),
        'time': t,
    },
)
da

We then construct our interface with a slider,
a node that slices our data at the index of the slider,
and a `figure3d`.

In [ ]:
# Use Plopp's widget to slice dimensions
slider = pw.SliceWidget(da, dims=['time'])
slider_node = pp.widget_node(slider)

slice_node = pw.slice_dims(data_array=da, slices=slider_node)

fig = pp.figure3d(slice_node, pixel_size=0.3)
pp.widgets.Box([fig, slider])

In [ ]:
pp.show_graph(fig)